In [22]:
from lpsim.server import Cards, Summons, Characters, CharacterStatus, TeamStatus
from lpsim.server.consts import IconType
from typing import get_type_hints
import time
import pydantic
import json

In [ ]:
# test parse speed
start_time = time.time()
for c in Cards.__args__:
    print(c.__name__)
    name_hints = get_type_hints(c)['name'].__args__
    print(name_hints)
print("--- %s seconds ---" % (time.time() - start_time))

In [30]:
# extract all names with their types
names = {}
unions = {
    'card': Cards, 
    'summon': Summons,
    'character': Characters,
    'character_status': CharacterStatus,
    'team_status': TeamStatus,
}
for key, union in unions.items():
    # if 'status' in key:
    #     key = 'status'
    # if key in ['character', 'summon', 'card']:
    #     key = 'cardface'
    for c in union.__args__:
        name_hints = get_type_hints(c)['name'].__args__
        for name in name_hints:
            name = f'{key}/{name}'
            if (
                name in names
                and get_type_hints(c)['type'] != get_type_hints(names[name])['type']
            ):
                print(f'conflict name: {name}')
            names[name] = c

In [32]:
name_to_icon = {}
for name in names:
    if 'status/' not in name:
        name_to_icon[name] = None
    else:
        self_name = name.split('/')[1]
        hints = {}
        # parent_classes = list(names[name].__mro__)
        # parent_classes.reverse()
        # for cls in parent_classes:
        #     hints.update(get_type_hints(cls))
        #     print(cls, hints)
        hints.update(get_type_hints(names[name]))
        # print(name, parent_classes)
        icon_hints = hints['icon_type'].__args__
        if len(icon_hints) != 1:
            args = {
                'name': self_name,
                'position': {
                    'player_idx': -1,
                    'area': 'INVALID',
                    'id': -1
                }
            }
            if 'mark' in hints:
                args['mark'] = hints['mark'].__args__[0]
            obj = pydantic.parse_obj_as(names[name], args)
            icon_hints = [obj.icon_type]
            if 'Midare Ranzan' in self_name:
                # for ranzan, need manually set
                last = self_name.split(' ')[-1]
                icon_hints = [
                    {
                        'New': IconType.SPECIAL,
                        'Ranzan': IconType.ELEMENT_ENCHANT_WIND,
                        'Pyro': IconType.ELEMENT_ENCHANT_FIRE,
                        'Hydro': IconType.ELEMENT_ENCHANT_WATER,
                        'Electro': IconType.ELEMENT_ENCHANT_ELEC,
                        'Cryo': IconType.ELEMENT_ENCHANT_ICE,
                    }[last]
                ]
            print(f'warning: {name} has multiple icon types, use {icon_hints[0]}')
        name_to_icon[name] = icon_hints[0]
len(name_to_icon)

441

In [33]:
print(json.dumps(name_to_icon))

{"card/Magic Guide": null, "card/Raven Bow": null, "card/Traveler's Handy Sword": null, "card/White Iron Greatsword": null, "card/White Tassel": null, "card/Sacrificial Fragments": null, "card/Sacrificial Greatsword": null, "card/Sacrificial Sword": null, "card/Sacrificial Bow": null, "card/A Thousand Floating Dreams": null, "card/Amos' Bow": null, "card/Elegy for the End": null, "card/Wolf's Gravestone": null, "card/The Bell": null, "card/Lithic Spear": null, "card/Vortex Vanquisher": null, "card/Engulfing Lightning": null, "card/Aquila Favonia": null, "card/Skyward Atlas": null, "card/Skyward Harp": null, "card/Skyward Pride": null, "card/Skyward Spine": null, "card/Skyward Blade": null, "card/Favonius Sword": null, "card/Fruit of Fulfillment": null, "card/King's Squire": null, "card/Moonpiercer": null, "card/Adventurer's Bandana": null, "card/Lucky Dog's Silver Circlet": null, "card/Traveling Doctor's Handkerchief": null, "card/Gambler's Earrings": null, "card/Instructor's Cap": nul